# Single Responsibility Principle
- Each class should only have one reason to change.
- Separation of concerns - different classes handling different, independent tasks/problems.

In [3]:
class Journal:
	def __init__(self) -> None:
			self.entries = []
			self.count = 0

	def add_entry(self, text: str) -> None:
		self.entries.append(f'{self.count}: {text}')
		self.count += 1

	def remove_entry(self, index: int) -> None:
		self.entries.pop(index)
		self.count -= 1
	
	def __str__(self) -> str:
		return '\n'.join(self.entries)

	# Break SRP
	def save(self, filename: str) -> None:
		with open(filename, 'w') as f:
			f.write(str(self))

	def load(self, filename: str) -> None:
		pass

	def load_from_web(self, url: str) -> None:
		pass

class PersistenceManager:
	@staticmethod
	def save_to_file(journal: Journal, filename: str) -> None:
		with open(filename, 'w') as f:
			f.write(str(journal))

j = Journal()
j.add_entry('I cried today.')
j.add_entry('I ate a bug.')
print(j)

PersistenceManager.save_to_file(j, 'journal.txt')

with open('journal.txt') as f:
	print(f.read())


0: I cried today.
1: I ate a bug.
0: I cried today.
1: I ate a bug.


# Open - Closed Principle
- Open for extension, Closed for modification
- Specification

In [ ]:
from enum import Enum
from typing import List

class Color(Enum):
	RED = 1
	GREEN = 2
	BLUE = 3

class Size(Enum):
	SMALL = 1
	MEDIUM = 2
	LARGE = 3

class Product:
	def __init__(self, name: str, color: Color, size: Size) -> None:
		self.name = name
		self.color = color
		self.size = size

	def __str__(self) -> str:
		return f'{self.name} is {self.color.name} and {self.size.name}'
	
class ProductFilter:
	def filter_by_color(self, products: List[Product], color: Color) -> List[Product]:
		for p in products:
			if p.color == color: yield p

	def filter_by_size(self, products: List[Product], size: Size) -> List[Product]:
		for p in products:
			if p.size == size: yield p

	def filter_by_size_and_color(self, products: List[Product], size: Size, color: Color) -> List[Product]:
		for p in products:
			if p.size == size and p.color == color:
				yield p

# Specification
class Specification:
	def is_satisfied(self, item: object) -> bool:
		pass

	def __and__(self, other: 'Specification') -> 'Specification':
		return AndSpecification(self, other)

class Filter:
	def filter(self, items: List, spec: Specification) -> List:
		pass

class ColorSpecification(Specification):
	def __init__(self, color: Color) -> None:
		self.color = color

	def is_satisfied(self, item: Product) -> bool:
		return item.color == self.color

class SizeSpecification(Specification):
	def __init__(self, size: Size) -> None:
		self.size = size

	def is_satisfied(self, item: Product) -> bool:
		return item.size == self.size

class AndSpecification(Specification):
	def __init__(self, *args: Specification.args) -> None:
		self.args = args

	def is_satisfied(self, item: Product) -> bool:
		return all(map(lambda s: s.is_satisfied(item), self.args))

class BetterFilter(Filter):
	def filter(self, items: List, spec: Specification) -> List:
		for item in items:
			if spec.is_satisfied(item):
				yield item

apple = Product('Apple', Color.GREEN, Size.SMALL)
tree = Product('Tree', Color.GREEN, Size.LARGE)
house = Product('House', Color.BLUE, Size.LARGE)
products = [apple, tree, house]

pf = ProductFilter()
print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
	print(p)

bf = BetterFilter()
print('Green products (new):')
green = ColorSpecification(Color.GREEN)
for p in bf.filter(products, green):
	print(p)

print('Large products (new):')
large = SizeSpecification(Size.LARGE)
for p in bf.filter(products, large):
	print(p)

print('Large blue items (new):')
# large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))
large_blue = large & ColorSpecification(Color.BLUE)
for p in bf.filter(products, large_blue):
	print(p)

# Liskov Substitution Principle
- If you have some interface that takes some sort of base class, you should be able to stick a derived class in there and everything should work
- You should be able to substitute a base type for a subtype

In [8]:
class Rectangle:
    def __init__(self, width, height):
        self._height = height
        self._width = width

    @property
    def area(self):
        return self._width * self._height

    def __str__(self):
        return f'Width: {self.width}, height: {self.height}'

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        self._height = value


class Square(Rectangle):
    def __init__(self, size):
        Rectangle.__init__(self, size, size)

    @Rectangle.width.setter
    def width(self, value):
        _width = _height = value

    @Rectangle.height.setter
    def height(self, value):
        _width = _height = value


def use_it(rc):
    w = rc.width
    rc.height = 10  # unpleasant side effect
    expected = int(w * 10)
    print(f'Expected an area of {expected}, got {rc.area}')


rc = Rectangle(2, 3)
use_it(rc)

sq = Square(5)
use_it(sq)


Expected an area of 20, got 20
Expected an area of 50, got 25


# Interface Segregation Principle
- Don't put too much into an interface; split into separate interfaces
- "You aren't gonna need it"(YAGNI) is a principle of extreme programming (XP) that states a programmer should not add functionality until deemed necessary.
- XP co-founder Ron Jeffries has written: "Always implement things when you actually need them, never when you just foresee that you need them."Other forms of the phrase include "You aren't going to need it" (YAGTNI) and "You ain't gonna need it" (YAGNI).

In [ ]:
from abc import abstractmethod

class Machine:
  def print(self, document):
    raise NotImplementedError()

  def fax(self, document):
    raise NotImplementedError()

  def scan(self, document):
    raise NotImplementedError()

# ok if you need a multifunction device
class MultiFunctionPrinter(Machine):
  def print(self, document):
    pass

  def fax(self, document):
    pass

  def scan(self, document):
    pass

class OldFashionedPrinter(Machine):
  def print(self, document):
    # ok - print stuff
    pass

  def fax(self, document):
    pass  # do-nothing

  def scan(self, document):
    """Not supported!"""
    raise NotImplementedError('Printer cannot scan!')

class Printer:
  @abstractmethod
  def print(self, document): pass

class Scanner:
  @abstractmethod
  def scan(self, document): pass

# same for Fax, etc.
class MyPrinter(Printer):
  def print(self, document):
    print(document)

class Photocopier(Printer, Scanner):
  def print(self, document):
    print(document)

  def scan(self, document):
    pass  # something meaningful

class MultiFunctionDevice(Printer, Scanner):  # , Fax, etc
  @abstractmethod
  def print(self, document):
    pass

  @abstractmethod
  def scan(self, document):
    pass

class MultiFunctionMachine(MultiFunctionDevice):
  def __init__(self, printer, scanner):
    self.printer = printer
    self.scanner = scanner

  def print(self, document):
    self.printer.print(document)

  def scan(self, document):
    self.scanner.scan(document)


printer = OldFashionedPrinter()
printer.fax(123)  # nothing happens
printer.scan(123)  # oops!


# Dependency Inversion Principle
- High level classes (or modules) should not depend on low level modules.
- Instead, they should depend on abstractions.

In [18]:
from abc import abstractmethod
from enum import Enum


class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2


class Person:
    def __init__(self, name):
        self.name = name


class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name): pass


class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name


class Research:
    # dependency on a low-level module directly
    # bad because strongly dependent on e.g. storage type

    # def __init__(self, relationships):
    #     # high-level: find all of john's children
    #     relations = relationships.relations
    #     for r in relations:
    #         if r[0].name == 'John' and r[1] == Relationship.PARENT:
    #             print(f'John has a child called {r[2].name}.')

    def __init__(self, browser):
        for p in browser.find_all_children_of("John"):
            print(f'John has a child called {p}')


parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris
John has a child called Matt


In [ ]:
# From thirdparty service
class PaypayAPI:
	def send_payment(self, amount): pass

class MoneyBookerAPI:
	@abstractmethod
	def pay(self, account): pass

class PaypalAPIAddapter(MoneyBookerAPI):
	def __init__(self) -> None:
		self.paypal = PaypayAPI()

	def pay(self, account):
		self.paypal.send_payment(account)

